### Testowanie uczenia się DFA z pomocą systemu przepisywania więzów

In [126]:
import copy, sys
import cProfile

sys.path.append("../")
sys.path.append("../inferring")
sys.path.append("../utils")
sys.path.append("../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.oracles.SRS as SRS

reload(SRS)
reload(Inferring)
reload(InferringDFA)
reload(utils.automats.DFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.oracles.SRS import SRS 

from inferring.InferringDFA import InferringDFA

In [127]:
def run_learning_process(test, oracle=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(test.dfa)
    learn_dfa = InferringDFA(_dfa, 
                                oracle, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    print(f"kontrprzykłady: liczba - {len(cnt_ex)}, długości - {cnt_ex}, cnt = {cnt}")
    return copy.deepcopy(dfa) 

In [128]:
class test_example():
    def __init__(self, dfa, descrip=''):
        self.dfa = copy.deepcopy(dfa)
        self.descrip = descrip

In [129]:
test_examples = []

dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1])),
                        dfa2=DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1])))
test = test_example(dfa=dfa_, descrip="plot, dfa1 - słowa zawierające literke 'a', dfa2 - słowa zawierające literke 'a'")
test_examples.append(test)
# result, without SRS at all: cnt = [69, 1]

dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=3, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):2, (2, 'b'):2}, F= set([2])), 
                        dfa2=DFA(Q=3, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):2, (2, 'b'):2}, F= set([2])))
test = test_example(dfa=dfa_, descrip="splot, dfa1 - wzorzec - 'bb', dfa2 - wzorzec 'aa'")
test_examples.append(test)
# result, without SRS at all: cnt = [289, 3]

dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F= set([3])),
                        dfa2=DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F= set([3])))
test = test_example(dfa=dfa_, descrip="splot, dfa1 - wzorzec 'bbb', dfa2 - wzorzec 'aaa'")
test_examples.append(test)
# result, without SRS at all: cnt = [800, 3]

dfa_ = DFA()
dfa_.create_convolution(dfa1 = DFA(Q=4, input_signs=['a', 'b'], δ = {(0, 'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):2, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F=set([3])),
                        dfa2 = DFA(Q=4, input_signs=['a', 'b'], δ = {(0, 'a'):0, (0, 'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3])))
test = test_example(dfa = dfa_, descrip = "splot, dfa1 - wzorzec 'aba', dfa2 - wzorzec 'bbb'")
test_examples.append(test)
# result, without SRS at all: cnt = [1388, 7]


dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3])),
                        dfa2=DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3])))
test = test_example(dfa=dfa_, descrip="splot, dfa1 - słowa zawierające i literę 'a' i literę 'b', dfa2 - słowa zawierające i literę 'a' i literę 'b'")
test_examples.append(test)
# result, without SRS at all: cnt = [751, 10]

dfa_ = DFA(Q=3, input_signs=['0', '1', '2', '3', '4', '5', '6', '7'], δ={(0,'0'):0, (0,'1'):2, (0,'2'):2, (0,'3'):0, (0,'4'):2, (0,'5'):0, (0,'6'):1, (0,'7'):2, (1,'0'):2, (1,'1'):0, (1,'2'):1, (1,'3'):2, (1,'4'):1, (1,'5'):2, (1,'6'):2, (1,'7'):1, (2,'0'):2, (2,'1'):2, (2,'2'):2, (2,'3'):2, (2,'4'):2, (2,'5'):2, (2,'6'):2, (2,'7'):2}, F=set([0]))
test = test_example(dfa=dfa_, descrip="zakodowane dodawania w systemie dwójkowym")
test_examples.append(test)
# result, without SRS at all: cnt = [201, 1]

dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=5, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):2, (3,'a'):1, (3,'b'):4, (4,'a'):4, (4,'b'):4}, F=set([2, 4])),
                        dfa2=DFA(Q=5, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):2, (3,'a'):1, (3,'b'):4, (4,'a'):4, (4,'b'):4}, F=set([2, 4])))
test = test_example(dfa=dfa_, descrip="splot, dfa1 - 2 wzorce 'aa' lub 'bb', dfa2 - 2 wzorce 'aa' lub 'bb'" )
test_examples.append(test)
# result, without SRS at all: cnt = [1437, 7]

dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8])),
                        dfa2=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8])))
test = test_example(dfa=dfa_, descrip="splot, dfa1 - 2 wzorce 'aa' i 'bb', dfa2 - 2 wzorce 'aa' i 'bb'")  
test_examples.append(test)
# result, without SRS at all: cnt = [20522, 28]

dfa_ = DFA()
pdfa1_, pdfa2_ = DFA(), DFA()
pdfa1_.create_pattern_dfa(input_signs=['a', 'b'], patterns=['aa', 'bb'], _type="AND")
pdfa2_.create_pattern_dfa(input_signs=['a', 'b', 'c'], patterns=['aa', 'bb', 'c'], _type="AND")

dfa_.create_convolution(dfa1=pdfa1_, dfa2=pdfa2_)

test = test_example(dfa=dfa_, descrip="splot, dfa1 - 2 wzorce: 'aa', 'bb', dfa2 - 3 wzorce: 'aa', 'bb', 'c'")
test_examples.append(test)
# result, without SRS at all: cnt = [121709, 37]

In [130]:
test = test_examples[-1] 
print(test.descrip)
# test.dfa.print_transitions()

# d = run_learning_process(copy.deepcopy(test), oracle=None, check_consistency=False, equiv_query_fashion="DFS", debug=True)
# d = run_learning_process(copy.deepcopy(test), oracle=SRS(), check_consistency=False, equiv_query_fashion="DFS")
d = run_learning_process(copy.deepcopy(test), oracle=SRS(), check_consistency=True, equiv_query_fashion="DFS")

splot, dfa1 - 2 wzorce: 'aa', 'bb', dfa2 - 3 wzorce: 'aa', 'bb', 'c'
kontrprzykłady: liczba - 45, długości - [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 9, 9, 9, 9, 11, 10, 10, 10, 10, 10, 10, 10, 10, 11, 9, 12, 9, 12, 11, 9, 9, 9, 9, 15], cnt = [20892, 7]


## Przykład dziwny

In [131]:
patterns1 = ['aaa','bba','cca','ddd']
patterns2 = ['aaa','bbb','ccc','ddd']

pdfa_ = DFA()
pdfa_.create_pattern_dfa(input_signs=['a', 'b', 'c', 'd'], patterns=patterns1, _type="OR")
test_1 = test_example(dfa=pdfa_)

pdfa_ = DFA()
pdfa_.create_pattern_dfa(input_signs=['a', 'b', 'c', 'd'], patterns=patterns2, _type="OR")

test_2 = test_example(dfa=pdfa_)

d1 = run_learning_process(test=test_1)
d2 = run_learning_process(test=test_2)

# kontrprzykłady: [3, 3, 3, 3], cnt = [289, 5]
# kontrprzykłady: [3, 3, 3, 3], cnt = [289, 5]

kontrprzykłady: liczba - 4, długości - [3, 3, 3, 3], cnt = [289, 5]
kontrprzykłady: liczba - 4, długości - [3, 3, 3, 3], cnt = [289, 5]


In [132]:
conv_dfa = DFA() 
conv_dfa.create_convolution(dfa1=d1, dfa2=d2)
d = run_learning_process(test=test_example(dfa=conv_dfa), oracle=SRS(), check_consistency=True, debug=False)
d = run_learning_process(test=test_example(dfa=conv_dfa), oracle=None, check_consistency=False, debug=False)

kontrprzykłady: liczba - 39, długości - [6, 6, 6, 6, 6, 6, 6, 12, 5, 8, 8, 6, 5, 8, 8, 6, 5, 8, 8, 6, 9, 12, 8, 8, 6, 6, 6, 6, 6, 14, 6, 6, 6, 6, 6, 6, 10, 6, 6], cnt = [24258, 4]
kontrprzykłady: liczba - 68, długości - [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], cnt = [69499, 69]


### Duże automaty - ropoznające kilkadziesiąt wzorców (docelowo)

In [133]:
import random 
def random_patterns(input_signs, n=5, max_length=10):
    patterns = []
    for _ in range(n):
        p = ''.join(random.choices(population=input_signs,k=random.randint(1, max_length)))
        patterns.append(p)
    return patterns

random_patterns(input_signs=['a', 'b', 'c', 'd'], max_length=8)

['ca', 'c', 'da', 'dbdccbd', 'babdcbba']

In [134]:
random.seed(25)

input_signs = ['a', 'b', 'c', 'd']
number_of_patterns = 4
max_pattern_length = 6 

patterns1, patterns2 = random_patterns(input_signs, number_of_patterns, max_pattern_length), random_patterns(input_signs, number_of_patterns, max_pattern_length)

dfa1 = DFA() 
dfa2 = DFA()

dfa1.create_pattern_dfa(input_signs=input_signs, patterns=patterns1)
dfa2.create_pattern_dfa(input_signs=input_signs, patterns=patterns2)

print(f"patterns1 = {patterns1}")
print(f"|Q1| = {dfa1.Q}")

print(f"patterns2 = {patterns2}")
print(f"|Q2| = {dfa2.Q}")

d1 = run_learning_process(test_example(dfa=copy.deepcopy(dfa1)))
d2 = run_learning_process(test_example(dfa=copy.deepcopy(dfa2)))

print(f"after learning |Q1| = {d1.Q}")
s1 = ''.join(patterns1)
ans1 = d1.route(s1)

s2 = ''.join(patterns2)
print(f"after learning |Q2| = {d2.Q}")
ans2 = d2.route(s2)

print(f"{ans1}, {ans2}")

patterns1 = ['ddad', 'cad', 'aca', 'c']
|Q1| = 160
patterns2 = ['dcdcda', 'ccbbcc', 'ccd', 'a']
|Q2| = 392
kontrprzykłady: liczba - 4, długości - [7, 8, 9, 11], cnt = [2543, 5]
kontrprzykłady: liczba - 6, długości - [12, 13, 14, 15, 16, 16], cnt = [6646, 7]
after learning |Q1| = 36
after learning |Q2| = 52
('ddadcadacac', 1), ('dcdcdaccbbccccda', 1)


In [135]:
dfa_ = DFA() 
dfa_.create_convolution(dfa1=copy.deepcopy(d1), dfa2=copy.deepcopy(d2))

dfa_.route(s1+s2.upper())
print(f"|Q_conv(d1,d2)| = {dfa_.Q}")

test = test_example(dfa=dfa_, descrip=f"splot, dfa1 - wzorce: {patterns1}, dfa2 - wzorce: {patterns2}")
print(test.descrip)

|Q_conv(d1,d2)| = 1872
splot, dfa1 - wzorce: ['ddad', 'cad', 'aca', 'c'], dfa2 - wzorce: ['dcdcda', 'ccbbcc', 'ccd', 'a']


In [ ]:
cProfile.run('run_learning_process(test, oracle=None, check_consistency=False, debug=False)', sort='time')

In [ ]:
cProfile.run('run_learning_process(test, oracle=SRS(), check_consistency=True, debug=False)', sort='time')

### Uczenie się splotu DFA - **debugowanie** 

In [60]:
import BitVector
bv = BitVector.BitVector(bitlist =[1, 1, 0, 1])
print(f"bv = {bv}, int(bv) = {int(bv)}")

bv_ = BitVector.BitVector(bitlist = [1])
bv = bv + bv_

print(f"bv = {bv}, int(bv) = {int(bv)}")

bv = 1101, int(bv) = 13
bv = 11011, int(bv) = 27


<!-- ### Uczenie się splotów - **debugowanie**  -->